<a href="https://colab.research.google.com/github/austinMLB/NLPExplainabilityProject/blob/master/mixtral_placeholder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview
Mixtral, quantized and configured to run on T4
Based on
https://www.kaggle.com/discussions/general/469175

In [1]:
#fixing numpy
!pip uninstall numpy --yes

Found existing installation: numpy 1.24.4
Uninstalling numpy-1.24.4:
  Successfully uninstalled numpy-1.24.4


In [1]:

!pip install numpy==1.24.4

In [2]:

# fix numpy in colab
import numpy
from IPython.display import clear_output

# fix triton in colab
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

!git clone https://github.com/dvmazur/mixtral-offloading.git --quiet
!cd mixtral-offloading && pip install -q -r requirements.txt
!huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo --quiet --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-demo

clear_output()



In [3]:
import sys

sys.path.append("mixtral-offloading")
import torch
from torch.nn import functional as F
from hqq.core.quantize import BaseQuantizeConfig
from huggingface_hub import snapshot_download
from IPython.display import clear_output
from tqdm.auto import trange
from transformers import AutoConfig, AutoTokenizer
from transformers.utils import logging as hf_logging

from src.build_model import OffloadConfig, QuantConfig, build_model
#Initialize model


hqq_aten package not installed. HQQBackend.ATEN backend will not work unless you install the hqq_aten lib in hqq/kernels.


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## Authorization
This block is un-needed.  However, ensure you
* have HF_TOKEN set.  This is the token you can getfrom Hugging Face
* have navigated to https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1 and accepted the terms.

In [4]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [5]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"
state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo"

config = AutoConfig.from_pretrained(quantized_model_name)

device = torch.device("cuda:0")

##### Change this to 5 if you have only 12 GB of GPU VRAM #####
#for 16GB
#offload_per_layer = 4
# for 12 GB
# offload_per_layer = 5
#for 40GB
offload_per_layer = 0
###############################################################

num_experts = config.num_local_experts

offload_config = OffloadConfig(
    main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
    offload_size=config.num_hidden_layers * offload_per_layer,
    buffer_size=4,
    offload_per_layer=offload_per_layer,
)


attn_config = BaseQuantizeConfig(
    nbits=4,
    group_size=64,
    quant_zero=True,
    quant_scale=True,
)
attn_config["scale_quant_params"]["group_size"] = 256


ffn_config = BaseQuantizeConfig(
    nbits=2,
    group_size=16,
    quant_zero=True,
    quant_scale=True,
)
quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)


model = build_model(
    device=device,
    quant_config=quant_config,
    offload_config=offload_config,
    state_path=state_path,)

/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Loading experts:   0%|          | 0/32 [00:00<?, ?it/s]

In [6]:
!pip install langchain torch transformers sentence-transformers accelerate
import transformers, torch
from transformers import pipeline
from langchain import HuggingFacePipeline


In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)#"mistralai/Mixtral-8x7B-v0.1")

In [8]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype=torch.float16,
    max_new_tokens=1024,
    device=device
)


In [9]:
from langchain import HuggingFacePipeline

llm = HuggingFacePipeline(
    pipeline = pipeline,
    model_kwargs={"temperature": 0.0, "max_new_tokens":256},
)


In [10]:
from langchain_core.prompts.prompt import PromptTemplate
from langchain import LLMChain
#template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
template = "Instruction:\n{instruction}\n\nResponse:\n"
#prompt = template.format(
#    instruction="I am a male, black student.  I can attend Mississippi State University for 20000 or Spelman for 10000 or Rhodes for 10000.  Which would better for me? Please just provide the school as an answer.",
#    response="",
#)
#messages = [
#  {"role": "user", "content": "[text] QUESTION: [question]"},
#  {"role": "assistant", "content": "You are a helpful bot giving guidence to the user"},
#]
#input = tokenizer.apply_chat_template(messages)
task_prompt_template = PromptTemplate(
    input_variables=["instruction"], template=template, output_key = "structured_task"
)
task_chain = LLMChain(
    llm=llm, prompt=task_prompt_template
)


In [13]:
question = {"instruction":"I am a male, black student. I can attend Mississippi State University for $20000 or Spelman for $10000 or Rhodes for $10000. Which would better for me? Please just provide the school as an answer."}


In [14]:
print(task_chain.run(question))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Instruction:
I am a male, black student. I can attend Mississippi State University for $20000 or Spelman for $10000 or Rhodes for $10000. Which would better for me? Please just provide the school as an answer.

Response:
Based on the information provided, it is not possible to determine which school would be the best fit for you. While cost is certainly an important consideration, there are many other factors to take into account when choosing a college, such as the quality of the academic programs, the availability of financial aid, the location, the campus culture, and the opportunities for extracurricular involvement. It would be wise to visit each of the schools, if possible, and speak with admissions counselors, current students, and faculty members to get a better sense of what each school has to offer. Ultimately, the best college for you will be the one that meets your academic, financial, and personal needs and goals.
